In [1]:
spark

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 4, 6, Finished, Available)

In [2]:
# Set up data configuration
blob_account_name = "marckvnonprodblob"
blob_container_name = "bigdata"
# read only
blob_sas_token = "?sv=2021-10-04&st=2023-10-04T01%3A42%3A59Z&se=2024-01-02T02%3A42%3A00Z&sr=c&sp=rlf&sig=w3CH9MbCOpwO7DtHlrahc7AlRPxSZZb8MOgS6TaXLzI%3D"

wasbs_base_url = (
    f"wasbs://{blob_container_name}@{blob_account_name}.blob.core.windows.net/"
)
spark.conf.set(
    f"fs.azure.sas.{blob_container_name}.{blob_account_name}.blob.core.windows.net",
    blob_sas_token,
)


StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 4, 7, Finished, Available)

In [3]:
# Read the whole Reddit data

comments_path = "reddit-parquet/comments/"
submissions_path = "reddit-parquet/submissions/"

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 4, 8, Finished, Available)

In [4]:
comments_df = spark.read.parquet(f"{wasbs_base_url}{comments_path}")
submissions_df = spark.read.parquet(f"{wasbs_base_url}{submissions_path}")

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 4, 9, Finished, Available)

In [5]:
submissions_df.rdd.getNumPartitions()

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 4, 10, Finished, Available)

3080

In [7]:
from pyspark.sql.functions import col, asc,desc
submissions_filtered = submissions_df.filter(col("subreddit").isin(["Minecraft", "Eldenring","Genshin_Impact","pokemongo","RocketLeagueExchange","VALORANT","DestinyTheGame","CallOfDutyMobile","FIFA","pokemon"]))

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 4, 12, Finished, Available)

In [8]:
submissions_filtered.printSchema()

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 4, 13, Finished, Available)

root
 |-- adserver_click_url: string (nullable = true)
 |-- adserver_imp_pixel: string (nullable = true)
 |-- archived: boolean (nullable = true)
 |-- author: string (nullable = true)
 |-- author_cakeday: boolean (nullable = true)
 |-- author_flair_css_class: string (nullable = true)
 |-- author_flair_text: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- brand_safe: boolean (nullable = true)
 |-- contest_mode: boolean (nullable = true)
 |-- created_utc: timestamp (nullable = true)
 |-- crosspost_parent: string (nullable = true)
 |-- crosspost_parent_list: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- approved_at_utc: string (nullable = true)
 |    |    |-- approved_by: string (nullable = true)
 |    |    |-- archived: boolean (nullable = true)
 |    |    |-- author: string (nullable = true)
 |    |    |-- author_flair_css_class: string (nullable = true)
 |    |    |-- author_flair_text: string (nullable = true)
 |    |    

In [10]:
# display a subset of columns
submissions_filtered.select("subreddit", "author", "title", "selftext", "num_comments", "score", "created_utc", "year", "month").show()

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 4, 15, Finished, Available)

+--------------------+--------------------+--------------------+--------------------+------------+-----+-------------------+----+-----+
|           subreddit|              author|               title|            selftext|num_comments|score|        created_utc|year|month|
+--------------------+--------------------+--------------------+--------------------+------------+-----+-------------------+----+-----+
|                FIFA|       Robert-Fulton|if you every get ...|                    |          12|    0|2023-02-22 21:07:53|2023|    2|
|           Minecraft|          Dogeisagod|     Texting a house|So I’m building a...|           3|    1|2023-02-22 21:08:06|2023|    2|
|            VALORANT|     Status_Bet_8034|       Killjoy Mains|Got bored and so ...|           2|    2|2023-02-22 21:08:12|2023|    2|
|           Minecraft|         Age_Natural|Trident skeleton ...|                    |           3|    3|2023-02-22 21:08:13|2023|    2|
|             pokemon|     OneWhoGetsBread|the P

In [11]:
comments_df.printSchema()

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 4, 16, Finished, Available)

root
 |-- author: string (nullable = true)
 |-- author_cakeday: boolean (nullable = true)
 |-- author_flair_css_class: string (nullable = true)
 |-- author_flair_text: string (nullable = true)
 |-- body: string (nullable = true)
 |-- can_gild: boolean (nullable = true)
 |-- controversiality: long (nullable = true)
 |-- created_utc: timestamp (nullable = true)
 |-- distinguished: string (nullable = true)
 |-- edited: string (nullable = true)
 |-- gilded: long (nullable = true)
 |-- id: string (nullable = true)
 |-- is_submitter: boolean (nullable = true)
 |-- link_id: string (nullable = true)
 |-- parent_id: string (nullable = true)
 |-- permalink: string (nullable = true)
 |-- retrieved_on: timestamp (nullable = true)
 |-- score: long (nullable = true)
 |-- stickied: boolean (nullable = true)
 |-- subreddit: string (nullable = true)
 |-- subreddit_id: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)



In [12]:
# this code saves the data as a csv just for illustration, use parquet for faster analytics operations.
df_save = submissions_filtered.limit(5000).select("subreddit", "author", "title", "selftext", "num_comments", "score", "created_utc", "year", "month").cache()

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 4, 17, Finished, Available)

In [17]:
import os
CSV_DIR = os.path.join("Users/zx111/fall-2023-reddit-project-team-11/data", "csv")
df_save.write.csv(f"{CSV_DIR}/top_10_pop_games_text.csv")

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 4, 22, Submitted, Running)

In [39]:
df_save.to_csv(f"{CSV_DIR}/top_10_pop_games_text.csv", index=False)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 4, 44, Finished, Available)

AttributeError: 'DataFrame' object has no attribute 'to_csv'

In [27]:
df_save.write.csv(f"../../data/csv/top_10_pop_games_text.csv")

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 4, 32, Finished, Available)

In [ ]:
import pandas as pd

In [25]:
df_check=pd.read_csv("Users/zx111/fall-2023-reddit-project-team-11/data/csv/top_10_pop_games_text.csv")

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 4, 30, Finished, Available)

FileNotFoundError: [Errno 2] No such file or directory: 'Users/zx111/fall-2023-reddit-project-team-11/data/csv/top_10_pop_games_text.csv'